# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount pearl,47.5166,-52.7813,33.46,96,100,24.16,CA,1707110014
1,1,port lincoln,-34.7333,135.8667,66.60,51,4,20.83,AU,1707110014
2,2,zliten,32.4674,14.5687,55.29,77,0,12.06,LY,1707110014
3,3,kayangel,8.0823,134.7173,80.53,84,100,18.43,PW,1707110015
4,4,hafar al-batin,28.4328,45.9708,46.98,61,0,4.61,SA,1707110015


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df[city_data_df["Max Temp"].between(-17, 80)]
ideal_city = ideal_city.loc[ideal_city["Humidity"]<40]
ideal_city = ideal_city.loc[ideal_city["Cloudiness"].between(20,80)]

# Drop any rows with null values
ideal_city.dropna(how="any")

# Display sample data
ideal_city.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
53,53,blackmans bay,-43.0167,147.3167,71.73,39,32,3.00,AU,1707110027
140,140,betare oya,5.6000,14.0833,67.48,26,48,5.68,CM,1707110048
142,142,bardai,11.6500,7.2333,60.67,17,56,15.21,NG,1707110049
192,192,rawson,-43.3002,-65.1023,77.79,28,43,30.69,AR,1707110063
255,255,taoudenni,22.6783,-3.9836,56.59,27,48,10.27,ML,1707110080


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]].reset_index(drop=True)
hotel_name = ""
hotel_df["Hotel Name"]=hotel_name

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,39,
1,betare oya,CM,5.6000,14.0833,26,
2,bardai,NG,11.6500,7.2333,17,
3,rawson,AR,-43.3002,-65.1023,28,
4,taoudenni,ML,22.6783,-3.9836,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 1
latitude = 0
longitude = 0
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
params = {"categories": categories,
        "limit": limit,
        "filters": filters,
        "bias": bias,
        "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.iloc[index,2]
    longitude = hotel_df.iloc[index,3]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
betare oya - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
rawson - nearest hotel: Hotel Deportivo
taoudenni - nearest hotel: No hotel found
bria - nearest hotel: No hotel found
wau - nearest hotel: Safari Park Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
ouesso - nearest hotel: Auberge Nianina
riti - nearest hotel: No hotel found
faya-largeau - nearest hotel: No hotel found
abeche - nearest hotel: No hotel found
nguigmi - nearest hotel: Guest PAM


,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,39,Villa Howden
1,betare oya,CM,5.6000,14.0833,26,No hotel found
2,bardai,NG,11.6500,7.2333,17,No hotel found
3,rawson,AR,-43.3002,-65.1023,28,Hotel Deportivo
4,taoudenni,ML,22.6783,-3.9836,27,No hotel found
5,bria,CF,6.5423,21.9863,31,No hotel found
6,wau,SS,7.7029,27.9953,21,Safari Park Hotel
7,new norfolk,AU,-42.7826,147.0587,38,Woodbridge on the Derwent
8,ouesso,CG,1.6136,16.0517,38,Auberge Nianina
9,riti,NG,7.9167,9.6167,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)